## To Dos

- transfer to OOP format
    - adhere to best practices for modularity
- handle continuous predictors
    - ^ + not necessarily split at all categorical levels of the split feature 
- regression
- multiple stopping conditions
- gini impurity option

In [1]:
import numpy as np
import pandas as pd

In [2]:
y = np.random.choice([1,0], size = (100,))

X = pd.DataFrame(np.random.choice([0, 1], size = (100, 5)), columns = list("ABCDE"))

In [4]:
def entropy(y):
    total = y.size
    value_counts = np.bincount(y).astype("float")
    proportions = value_counts / y.size
    
    return sum(-i * np.log(i) for i in proportions if i)

In [17]:
def classify(series, tree):
    feature = tree[0]
    subtree = tree[1]
    
    answer = series[feature]
    response = subtree[answer]
    
    if type(response) != list: #base case
        return subtree[answer]
    else:
        return classify(series, response) #recursive case
    
def ClassifyFrame(frame, tree):
    yhat = frame.apply(lambda x: classify(x, tree), axis = 1)
    return yhat

In [9]:
def gain(X, y, column):
    prior_entropy = entropy(y)
    total = y.size
    
    values = X[column].unique()
    proportions = X[column].value_counts() / total
    return prior_entropy - sum(proportions[i] * \
               entropy(y[np.array(X[column]) == i]) for i in values)

In [34]:
def GrowTree(X, y, candidates = None):
    
    if not candidates: # all columns are candidates2
        candidates = X.columns.tolist()   
    #should this be a leaf node?
    unique = np.unique(y)
    counts = np.bincount(y)
    nonzero_ind = np.nonzero(counts)[0]
    
    if nonzero_ind.size == 1:
        return unique[0] #base case: return pure outcome
    elif len(candidates) == 1:
        return unique[np.argmax(counts)] #base case: return mode outcome
    
    #choose a feature to split on
    gains = zip(candidates, map(lambda x: gain(X, y, x), candidates))
    split_feature = sorted(gains,  # choose feature with largest info gain 
                           key = lambda x: x[1], 
                           reverse = True)[0][0]
    
    #prepare new candidates for recursive call
    new_candidates = [i for i in candidates if i != split_feature]
    levels = X[split_feature].unique().tolist()
    
    subtree = {}
    
    for level in levels:
        
        X_with_level = X.loc[X[split_feature] == level, :] 
        y_with_level = y[np.array(X[split_feature] == level)]
        subtree[level] = GrowTree(X_with_level, 
                                   y_with_level, 
                                   new_candidates) #recursive call
        
    return [split_feature, subtree]
       

In [12]:
built_tree = GrowTree(X, y)
built_tree

['C',
 {0: ['A',
   {0: ['E', {0: ['B', {0: 1, 1: 0}], 1: ['B', {0: 0, 1: 0}]}],
    1: ['B', {0: ['D', {0: 0, 1: 0}], 1: ['D', {0: 1, 1: 1}]}]}],
  1: ['B',
   {0: ['E', {0: ['A', {0: 1, 1: 0}], 1: ['A', {0: 0, 1: 0}]}],
    1: ['E', {0: ['D', {0: 0, 1: 0}], 1: ['A', {0: 0, 1: 0}]}]}]}]

In [25]:
yhat = ClassifyFrame(X, built_tree)
X["y"] = y

In [37]:
np.mean(y == np.random.choice([0,1]))

0.48999999999999999